## Higher Order Functions

In [0]:
%sql
create or replace temp view order_items as 
select * from 
values
 (1, array ('smartphone', 'laptop', 'monitor')),
 (2, array('tablet' , 'headphones' , 'smartwatch')),
 (3, array('keyboarde', 'mouse'))
as order (order_id, items);

In [0]:
%sql
select * from order_items

order_id,items
1,"List(smartphone, laptop, monitor)"
2,"List(tablet, headphones, smartwatch)"
3,"List(keyboarde, mouse)"


#### 1-Convert all the item names to be UPPERCASE (TRANSFORM FUNCTION)

In [0]:
%sql
select order_id,
       TRANSFORM(items, x-> upper (X)) AS upper_items
from order_items;

order_id,upper_items
1,"List(SMARTPHONE, LAPTOP, MONITOR)"
2,"List(TABLET, HEADPHONES, SMARTWATCH)"
3,"List(KEYBOARDE, MOUSE)"


#### 2-Filter only items that contains the string 'smart' (Filter Function)

In [0]:
%sql
select order_id,
       FILTER(items, x-> x like '%smart%') AS smart_items
from order_items;

order_id,smart_items
1,List(smartphone)
2,List(smartwatch)
3,List()


#### 3-Check to see whether the order includes any 'monitor' (EXISTS Function)

In [0]:
%sql
select order_id,
       EXISTS(items, x-> x= 'monitor') AS has_monitor
from order_items;

order_id,has_monitor
1,true
2,false
3,false


## Array with more than one object

In [0]:
%sql
create  or replace temp view order_items as 
select * from values 
  (1, array( 
       named_struct ('name', 'smartphone', 'price', 699),
       named_struct ('name', 'laptop', 'price', 1199),
       named_struct ('name', 'monitor', 'price', 399)
     )),
  (2, array(
       named_struct ('name', 'tablet' , 'price', 599),
       named_struct ('name', 'headphones' , 'price', 199),
       named_struct ('name', 'smartwatch', 'price', 299)
     )),
  (3, array(
       named_struct ('name', 'keyboarde', 'price', 89),
       named_struct ('name', 'mouse', 'price', 59)
     ))
as orders (order_id, items);



In [0]:
%sql
select * from order_items;

order_id,items
1,"List(List(smartphone, 699), List(laptop, 1199), List(monitor, 399))"
2,"List(List(tablet, 599), List(headphones, 199), List(smartwatch, 299))"
3,"List(List(keyboarde, 89), List(mouse, 59))"


#### 1-Convert all the item names to be UPPERCASE & add 10% TAX to each item (TRANSFORM Function)

In [0]:
%sql
select order_id,
       Transform (items, x-> named_struct(
                                           'name' , UPPER(x.name),
                                           'price', ROUND(x.price * 1.10, 2)
                                            )) items_with_tax
  from order_items;

order_id,items_with_tax
1,"List(List(SMARTPHONE, 768.90), List(LAPTOP, 1318.90), List(MONITOR, 438.90))"
2,"List(List(TABLET, 658.90), List(HEADPHONES, 218.90), List(SMARTWATCH, 328.90))"
3,"List(List(KEYBOARDE, 97.90), List(MOUSE, 64.90))"


#### 2-Calculate the total order amount for each of the order (AGGREGATE Function)

In [0]:
%sql
select order_id,
       aggregate(items, 0,(acc,x) -> acc+ x.price) as total_order_price
  from order_items;

order_id,total_order_price
1,2297
2,1097
3,148


## Map Function

In [0]:
%sql
create or replace temp view order_items_prices as 
select * from values
 (1, map('smartphone', 699, 'laptop', 1199, 'monitor', 399)),
 (2, map('tablet', 599  , 'headphones', 199, 'smartwatch', 299)),
 (3, map('keyboarde', 89, 'mouse', 59) )
as orders (order_id, items_prices);

In [0]:
%sql
select * from order_items_prices;
    

order_id,items_prices
1,"Map(smartphone -> 699, laptop -> 1199, monitor -> 399)"
2,"Map(tablet -> 599, headphones -> 199, smartwatch -> 299)"
3,"Map(keyboarde -> 89, mouse -> 59)"


#### 1- Convert all the item names to be UPPERCASE (TRANSFORM fUNCTION)

In [0]:
%sql
SELECT order_id,
       Transform_keys (items_prices, (item, price) -> UPPER(item)) AS items_upper_case
 from order_items_prices;

order_id,items_upper_case
1,"Map(SMARTPHONE -> 699, LAPTOP -> 1199, MONITOR -> 399)"
2,"Map(TABLET -> 599, HEADPHONES -> 199, SMARTWATCH -> 299)"
3,"Map(KEYBOARDE -> 89, MOUSE -> 59)"


#### 2-Apply 10% TAX to item prices (TRANSFORM_VALUES Function)

In [0]:
%sql
-- Round (2 decimals)
select order_id,
       Transform_values(items_prices, (item,price) -> ROUND(price*1.10, 2)) as items_with_tax
from order_items_prices;

order_id,items_with_tax
1,"Map(smartphone -> 768.90, laptop -> 1318.90, monitor -> 438.90)"
2,"Map(tablet -> 658.90, headphones -> 218.90, smartwatch -> 328.90)"
3,"Map(keyboarde -> 97.90, mouse -> 64.90)"


#### 3-Filter items with price above $500 (MAP_FILTER Function)

In [0]:
%sql
select order_id,
       MAP_FILTER(items_prices, (item,price) -> price>500) as premium_items
from order_items_prices;

order_id,premium_items
1,"Map(smartphone -> 699, laptop -> 1199)"
2,Map(tablet -> 599)
3,Map()
